import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, lit
from pyspark.sql.functions import max, avg, min, desc
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when

print(pyspark.__version__)

3.5.3


create local SparkSession

In [2]:
spark = SparkSession.builder.appName("SimplePySparkJob").getOrCreate()

read csv with inferschema

In [3]:
input_file = 'ds_salaries.csv'
df = spark.read.csv(input_file, header=True, inferSchema=True)

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [5]:
df = spark.read.csv(input_file, header=True, inferSchema=True)

write schema of scv on screen

In [6]:
schema = df.schema


create schema of this scv

In [7]:
spark.stop()
spark = SparkSession.builder.appName("SimplePySparkJob").getOrCreate()

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [8]:
input_file = 'ds_salaries.csv'
df = spark.read.schema(schema).csv(input_file)

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [9]:
df.schema

StructType([StructField('_c0', IntegerType(), True), StructField('work_year', IntegerType(), True), StructField('experience_level', StringType(), True), StructField('employment_type', StringType(), True), StructField('job_title', StringType(), True), StructField('salary', IntegerType(), True), StructField('salary_currency', StringType(), True), StructField('salary_in_usd', IntegerType(), True), StructField('employee_residence', StringType(), True), StructField('remote_ratio', IntegerType(), True), StructField('company_location', StringType(), True), StructField('company_size', StringType(), True)])

now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [12]:
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

print data in dataframe using display(df.toPandas())

In [10]:
display(df.toPandas())

,_c0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [11]:
df_job_titles = df.select('job_title').distinct()

print all rows from df_job_titles without truncating jobs

In [12]:
for row in df_job_titles.toLocalIterator():
    print(row)

Row(job_title='3D Computer Vision Researcher')
Row(job_title='Lead Data Engineer')
Row(job_title='Head of Machine Learning')
Row(job_title='Data Specialist')
Row(job_title='Data Analytics Lead')
Row(job_title='Machine Learning Scientist')
Row(job_title='Lead Data Analyst')
Row(job_title='Data Engineering Manager')
Row(job_title='Staff Data Scientist')
Row(job_title='ETL Developer')
Row(job_title='Director of Data Engineering')
Row(job_title='Product Data Analyst')
Row(job_title='Principal Data Scientist')
Row(job_title='AI Scientist')
Row(job_title='Director of Data Science')
Row(job_title='Machine Learning Engineer')
Row(job_title='Lead Data Scientist')
Row(job_title='Machine Learning Infrastructure Engineer')
Row(job_title='Data Science Engineer')
Row(job_title='Machine Learning Manager')
Row(job_title='Research Scientist')
Row(job_title='Head of Data')
Row(job_title='Cloud Data Engineer')
Row(job_title='Machine Learning Developer')
Row(job_title='Data Scientist')
Row(job_title='Fina

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [13]:
df_analytic = df.groupBy('job_title').agg(
    avg('salary_in_usd').alias('avg_salary'),
    min('salary_in_usd').alias('min_salary'),
    max('salary_in_usd').alias('max_salary'),
)

print all rows from df_analytic without trancating jobs

In [14]:
for row in df_analytic.toLocalIterator(): print(row)

Row(job_title='3D Computer Vision Researcher', avg_salary=5409.0, min_salary=5409, max_salary=5409)
Row(job_title='Lead Data Engineer', avg_salary=139724.5, min_salary=56000, max_salary=276000)
Row(job_title='Head of Machine Learning', avg_salary=79039.0, min_salary=79039, max_salary=79039)
Row(job_title='Data Specialist', avg_salary=165000.0, min_salary=165000, max_salary=165000)
Row(job_title='Data Analytics Lead', avg_salary=405000.0, min_salary=405000, max_salary=405000)
Row(job_title='Machine Learning Scientist', avg_salary=158412.5, min_salary=12000, max_salary=260000)
Row(job_title='Lead Data Analyst', avg_salary=92203.0, min_salary=19609, max_salary=170000)
Row(job_title='Data Engineering Manager', avg_salary=123227.2, min_salary=59303, max_salary=174000)
Row(job_title='Staff Data Scientist', avg_salary=105000.0, min_salary=105000, max_salary=105000)
Row(job_title='ETL Developer', avg_salary=54957.0, min_salary=54957, max_salary=54957)
Row(job_title='Director of Data Engineerin

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [15]:
wind_spec = Window.orderBy(desc('avg_salary'))
df_analytic = df_analytic.withColumn('row_id', row_number().over(wind_spec))

print all data from df_analytic

In [16]:
df_analytic.show()

+--------------------+------------------+----------+----------+------+
|           job_title|        avg_salary|min_salary|max_salary|row_id|
+--------------------+------------------+----------+----------+------+
| Data Analytics Lead|          405000.0|    405000|    405000|     1|
|Principal Data En...| 328333.3333333333|    185000|    600000|     2|
|Financial Data An...|          275000.0|    100000|    450000|     3|
|Principal Data Sc...|215242.42857142858|    148261|    416000|     4|
|Director of Data ...|          195074.0|    130026|    325000|     5|
|      Data Architect| 177873.9090909091|     90700|    266400|     6|
|Applied Data Scie...|          175655.0|     54238|    380000|     7|
|  Analytics Engineer|          175000.0|    135000|    205300|     8|
|     Data Specialist|          165000.0|    165000|    165000|     9|
|        Head of Data|          160162.6|     32974|    235000|    10|
|Machine Learning ...|          158412.5|     12000|    260000|    11|
|Data 

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [17]:
df_analytic = df_analytic.select('row_id', *tuple(df_analytic.columns[:-1]))

print df_analytic now

In [18]:
df_analytic.show()

+------+--------------------+------------------+----------+----------+
|row_id|           job_title|        avg_salary|min_salary|max_salary|
+------+--------------------+------------------+----------+----------+
|     1| Data Analytics Lead|          405000.0|    405000|    405000|
|     2|Principal Data En...| 328333.3333333333|    185000|    600000|
|     3|Financial Data An...|          275000.0|    100000|    450000|
|     4|Principal Data Sc...|215242.42857142858|    148261|    416000|
|     5|Director of Data ...|          195074.0|    130026|    325000|
|     6|      Data Architect| 177873.9090909091|     90700|    266400|
|     7|Applied Data Scie...|          175655.0|     54238|    380000|
|     8|  Analytics Engineer|          175000.0|    135000|    205300|
|     9|     Data Specialist|          165000.0|    165000|    165000|
|    10|        Head of Data|          160162.6|     32974|    235000|
|    11|Machine Learning ...|          158412.5|     12000|    260000|
|    1

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [19]:
biggest_sal_df = df.groupBy('experience_level').agg(
    max('salary_in_usd').alias('biggest_salary'),
)

biggest_sal_df.show()

df_exp_lvl = df.join(
    biggest_sal_df,
    (df.experience_level == biggest_sal_df.experience_level) &
    (df.salary_in_usd == biggest_sal_df.biggest_salary),
    how='inner'
)

+----------------+--------------+
|experience_level|biggest_salary|
+----------------+--------------+
|              EX|        600000|
|              MI|        450000|
|              EN|        250000|
|              SE|        412000|
+----------------+--------------+



print here df_exp_lvl

In [20]:
df_exp_lvl.show()

+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+----------------+--------------+
|_c0|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|experience_level|biggest_salary|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+----------------+--------------+
| 33|     2020|              MI|             FT|  Research Scientist|450000|            USD|       450000|                US|           0|              US|           M|              MI|        450000|
| 37|     2020|              EN|             FT|Machine Learning ...|250000|            USD|       250000|                US|          50|              US|           L|              EN|        250

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [21]:
df_best = df.alias('df').join(
    biggest_sal_df.alias('bg_sal'),
    (df.experience_level == biggest_sal_df.experience_level),
    how='inner'
).filter(col('df.salary_in_usd') == col('bg_sal.biggest_salary')).select(
    'df._c0', 'df.experience_level', 'bg_sal.biggest_salary', 'df.employee_residence',
)

print df_best

In [22]:
df_best.show()

+---+----------------+--------------+------------------+
|_c0|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 33|              MI|        450000|                US|
| 37|              EN|        250000|                US|
| 63|              SE|        412000|                US|
| 97|              MI|        450000|                US|
|252|              EX|        600000|                US|
+---+----------------+--------------+------------------+



drop duplicates if exist by experience_level

In [23]:
df_best = df_best.dropDuplicates(['experience_level',])

print df_best

In [24]:
df_best.show()

+---+----------------+--------------+------------------+
|_c0|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|              EN|        250000|                US|
|252|              EX|        600000|                US|
| 33|              MI|        450000|                US|
| 63|              SE|        412000|                US|
+---+----------------+--------------+------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [25]:
df_new_best = df_best.withColumns({
    "experience_level": when(df_best.experience_level == 'MI', 'middle').\
        when(df_best.experience_level == 'SE', 'senior').otherwise(lit(None)),
})

print df_new_best

In [26]:
df_new_best.show()

+---+----------------+--------------+------------------+
|_c0|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|            NULL|        250000|                US|
|252|            NULL|        600000|                US|
| 33|          middle|        450000|                US|
| 63|          senior|        412000|                US|
+---+----------------+--------------+------------------+



write df_new_best like 1.csv and load then it to df_final

In [30]:
df_new_best.toPandas().to_csv(
    '1.csv',
    index=False,
    encoding='utf-8',
)

df_final = spark.read.csv('1.csv', header=True, inferSchema=True)


print df_final

In [35]:
df_final.show()

+---+----------------+--------------+------------------+
|_c0|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|            NULL|        250000|                US|
|252|            NULL|        600000|                US|
| 33|          middle|        450000|                US|
| 63|          senior|        412000|                US|
+---+----------------+--------------+------------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [40]:
df_final = df_final.dropna()
df_final = df_final.alias('df_f').join(
    df.alias('df'),
    (df.salary_in_usd == df_final.biggest_salary) &
    (df.employee_residence == df_final.employee_residence),
    how='inner'
)

print df_final

In [41]:
df_final.show()

+---+----------------+--------------+------------------+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|experience_level|biggest_salary|employee_residence|_c0|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+----------------+--------------+------------------+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
| 33|          middle|        450000|                US| 33|     2020|              MI|             FT|  Research Scientist|450000|            USD|       450000|                US|           0|              US|           M|
| 63|          senior|        412000|                US| 63|     2020|              SE|             FT| 

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [53]:
biggest_salary_in_usd = df_final.agg(max('salary_in_usd').alias('biggest_salary')).collect()[0]['biggest_salary']
biggest_salary_in_usd

450000

It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system